In [76]:
import pandas as pd
import utils
import datetime as dt
from dateutil.parser import *

In [77]:
df_trades =  pd.read_pickle("USD_JPY_H4_trades.pkl")

In [78]:
pair = 'USD_JPY'

In [79]:
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, "M5"))

In [80]:
df_raw.shape

(416683, 14)

In [81]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [82]:
df_trades["time"] = [parse(x) for x in df_trades.time]
df_raw["time"] = [parse(x) for x in df_raw.time]

In [83]:
#df_trades.head()
df_trades['next'] = df_trades['time'].shift(-1)

In [84]:
df_trades['trade_end'] = df_trades.next + dt.timedelta(hours=3, minutes=55)
df_trades['trade_start'] = df_trades.time + dt.timedelta(hours=4)

In [85]:
df_trades[['time', 'next', 'trade_start', 'trade_end']].head()

,time,next,trade_start,trade_end
2,2019-01-02 10:00:00+00:00,2019-01-03 02:00:00+00:00,2019-01-02 14:00:00+00:00,2019-01-03 05:55:00+00:00
6,2019-01-03 02:00:00+00:00,2019-01-03 18:00:00+00:00,2019-01-03 06:00:00+00:00,2019-01-03 21:55:00+00:00
10,2019-01-03 18:00:00+00:00,2019-01-03 22:00:00+00:00,2019-01-03 22:00:00+00:00,2019-01-04 01:55:00+00:00
11,2019-01-03 22:00:00+00:00,2019-01-04 06:00:00+00:00,2019-01-04 02:00:00+00:00,2019-01-04 09:55:00+00:00
13,2019-01-04 06:00:00+00:00,2019-01-04 18:00:00+00:00,2019-01-04 10:00:00+00:00,2019-01-04 21:55:00+00:00


In [86]:
df_trades.dropna(inplace=True)
df_trades.reset_index(drop=True, inplace=True)

In [90]:
def signal_text(signal):
    if signal == 1:
        return 'BUY'
    elif signal == -1:
        return 'SELL'
    return 'NONE'

def triggered(direction, current_price, signal_price):
    if direction == 1 and current_price > signal_price:
        return True
    elif direction == -1 and current_price < signal_price:
        return True
    return False

def end_hit_calc(direction, SL, price, start_price):
    # STOP LOSS price @100, ENTRY PRICE @200, CURRENT PRICE @150, TAKEPROFIT $400
    '''CHANGE IN PRICE WHEN HIT STOP LOSS = -100 = -1.0'''
    #-50 CHANGE IN PRICE = -0.5

    delta = price - start_price
    full_delta = start_price - SL
    fraction = abs(delta/full_delta)
    
    if direction == 1 and price>=start_price: 
        '''BUY'''
        #we're winning
        return fraction
    elif direction == 1 and price<start_price: 
        '''BUY'''
        #we're losing as we're below our start price
        return -fraction
    elif direction == -1 and price<=start_price: 
        '''SELL'''
        #we're winning
        return fraction
    elif direction == -1 and price>start_price: 
        '''SELL'''
        #we're losing 
        return -fraction

def process_buy(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price >= TP:
            return 2.0
        elif price <= SL:
            return -1.0
    return 0.0

def process_sell(start_index, TP, SL, prices, start_price):
    for price in prices[start_index:]:
        if price <= TP:
            return 2.0
        elif price >= SL:
            return -1.0
    return 0.0

def process_trade(start_index, direction, TP, SL, prices, start_price):
    if direction == 1:
        return process_buy(start_index, TP, SL, prices, start_price)
    else:
        return process_sell(start_index, TP, SL, prices, start_price)


def process_m5(m5_df, row):
    result = 0.0
    for index, price in enumerate(m5_df.mid_c.values):
        if triggered(row.SIGNAL, price, row.ENTRY) == True:
            #print(f"Signal at {index} {price:.2f} {row.ENTRY:.2f} {row.SIGNAL:.2f}")
            result = process_trade(index, row.SIGNAL, row.TAKEPROFIT, row.STOPLOSS, m5_df.mid_c.values, row.ENTRY)
            break
    return result

In [91]:
total = 0
for index, row in df_trades.iterrows():
    m5_data = df_raw[(df_raw.time >= row.time) & (df_raw.time <= row.trade_end)]
   #CHECK
    #print(f"{row.time} {signal_test(row.SIGNAL)} {row.ENTRY:.2f} {m5_data.shape}")
    total += process_m5(m5_data, row)
    #if index>20:
    #    break
print(total)

199.0
